In [2]:
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.axes import Axes
import matplotlib.gridspec as gridspec
import numpy as np
from typing import cast
d = np.load("output_keypoints_3d.npz")

In [3]:
kps = d["reconstruction"]
kps.shape

(167, 17, 3)

In [4]:

def show3Dpose(vals: NDArray, ax: Axes3D, fix_z: bool):
    ax.view_init(elev=15., azim=70)
    colors = [
        (138 / 255, 201 / 255, 38 / 255),  # green, spine & head
        (255 / 255, 202 / 255, 58 / 255),  # yellow, arms & shoulders
        (25 / 255, 130 / 255, 196 / 255),  # blue, legs & hips
    ]

    I = np.array([0, 0, 1, 4, 2, 5, 0, 7, 8, 8, 14, 15, 11, 12, 8, 9])
    J = np.array([1, 4, 2, 5, 3, 6, 7, 8, 14, 11, 15, 16, 12, 13, 9, 10])

    LR = [3, 3, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1]

    for i in np.arange(len(I)):
        x, y, z = [np.array([vals[I[i], j], vals[J[i], j]]) for j in range(3)]
        ax.plot(x, y, z, lw=3, color=colors[LR[i] - 1])

    RADIUS = 0.72

    xroot, yroot, zroot = vals[0, 0], vals[0, 1], vals[0, 2]
    ax.set_xlim3d((-RADIUS + xroot, RADIUS + xroot))
    ax.set_ylim3d((-RADIUS + yroot, RADIUS + yroot))

    if fix_z:
        left_z = max(0.0, -RADIUS + zroot)
        right_z = RADIUS + zroot
        ax.set_zlim3d([left_z, right_z])
        # ax.set_zlim3d([0, 1.5])
    else:
        ax.set_zlim3d([-RADIUS + zroot, RADIUS + zroot])

    ax.set_aspect('equal')  # works fine in matplotlib==2.2.2 or 3.7.1

    white = (1.0, 1.0, 1.0, 0.0)
    ax.xaxis.set_pane_color(white)  # type: ignore
    ax.yaxis.set_pane_color(white)  # type: ignore
    ax.zaxis.set_pane_color(white)

    ax.tick_params('x', labelbottom=False)
    ax.tick_params('y', labelleft=False)
    ax.tick_params('z', labelleft=False)  # type: ignore

```json
"keypoints": { 0: "nose", 1: "left_eye", 2: "right_eye", 3: "left_ear", 4: "right_ear", 5: "left_shoulder", 6: "right_shoulder", 7: "left_elbow", 8: "right_elbow", 9: "left_wrist", 10: "right_wrist", 11: "left_hip", 12: "right_hip", 13: "left_knee", 14: "right_knee", 15: "left_ankle", 16: "right_ankle" }
```

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from typeguard import typechecked
from jaxtyping import Int, Float, Bool, Num, jaxtyped
from pydantic import BaseModel
from typing import Optional, Union

NDArray = np.ndarray
number =Union[int, float] 
Color = tuple[int, int, int]|str

class Joint(BaseModel):
    index:int
    opposite_index:Optional[int] = None
    name:str
    color:Color

# for coco
coco_joints = [
    Joint(index=0, name="nose", color=(255, 0, 0)),
    Joint(index=1, name="left_eye", opposite_index=2, color=(0, 255, 0)),
    Joint(index=2, name="right_eye", opposite_index=1, color=(0, 0, 255)),
    Joint(index=3, name="left_ear", opposite_index=4, color=(255, 255, 0)),
    Joint(index=4, name="right_ear", opposite_index=3, color=(0, 255, 255)),
    Joint(index=5, name="left_shoulder", opposite_index=6, color=(255, 0, 255)),
    Joint(index=6, name="right_shoulder", opposite_index=5, color=(255, 255, 255)),
    Joint(index=7, name="left_elbow", opposite_index=8, color=(0, 0, 0)),
    Joint(index=8, name="right_elbow", opposite_index=7, color=(100, 100, 100)),
    Joint(index=9, name="left_wrist", opposite_index=10, color=(200, 200, 200)),
    Joint(index=10, name="right_wrist", opposite_index=9, color=(150, 150, 150)),
    Joint(index=11, name="left_hip", opposite_index=12, color=(50, 50, 50)),
    Joint(index=12, name="right_hip", opposite_index=11, color=(100, 100, 100)),
    Joint(index=13, name="left_knee", opposite_index=14, color=(150, 150, 150)),
    Joint(index=14, name="right_knee", opposite_index=13, color=(200, 200, 200)),
    Joint(index=15, name="left_ankle", opposite_index=16, color=(250, 250, 250)),
    Joint(index=16, name="right_ankle", opposite_index=15, color=(255, 255, 255)),
]

# https://plotly.com/python-api-reference/generated/plotly.graph_objects.scatter3d.marker.html
# plotly.graph_objects.scatter3d.Marker
def to_rgb_str(color:tuple[int, int, int])->str:
    return f"rgb({color[0]},{color[1]},{color[2]})"
COLOR_SPINE = to_rgb_str((138, 201, 38)) # green, spine & head
COLOR_ARMS = to_rgb_str((255, 202, 58)) # yellow, arms & shoulders
COLOR_LEGS = to_rgb_str((25, 130, 196)) # blue, legs & hips
CIRCLE_SIZE = 6

# https://github.com/lllyasviel/ControlNet/discussions/266
# for human 3.6
human_36_joints = [
    Joint(index=0, name="bottom_torso", color=COLOR_SPINE),
    Joint(index=1, name="left_hip", opposite_index=4, color=COLOR_LEGS),
    Joint(index=2, name="left_knee", opposite_index=5, color=COLOR_LEGS),
    Joint(index=3, name="left_foot", opposite_index=6, color=COLOR_LEGS),
    Joint(index=4, name="right_hip", opposite_index=1, color=COLOR_LEGS),
    Joint(index=5, name="right_knee", opposite_index=2, color=COLOR_LEGS),
    Joint(index=6, name="right_foot", opposite_index=3, color=COLOR_LEGS),
    Joint(index=7, name="center_torso", color=COLOR_SPINE),
    Joint(index=8, name="upper_torso", color=COLOR_SPINE),
    Joint(index=9, name="neck_base", color=COLOR_SPINE),
    Joint(index=10, name="center_head", color=COLOR_SPINE),
    Joint(index=11, name="right_shoulder", opposite_index=14, color=COLOR_ARMS),
    Joint(index=12, name="right_elbow", opposite_index=15, color=COLOR_ARMS),
    Joint(index=13, name="right_hand", opposite_index=16, color=COLOR_ARMS),
    Joint(index=14, name="left_shoulder", opposite_index=11, color=COLOR_ARMS),
    Joint(index=15, name="left_elbow", opposite_index=12, color=COLOR_ARMS),
    Joint(index=16, name="left_hand", opposite_index=13, color=COLOR_ARMS),
]

# note that the keypoints are in the format of (x, y, z)
# where x is the horizontal axis, z is the depth axis, and y is the vertical axis (upwards)
# i.e. the x-z plane is the ground plane

@jaxtyped(typechecker=typechecked)
def xz_ground_2_xy_ground(xyz: Num[NDArray, "17 3"]) -> Num[NDArray, "17 3"]:
    """
    essentially make xyz -> xzy
    """
    return np.array([xyz[:, 0], xyz[:, 2], xyz[:, 1]]).T

def swap_axes_direction(arr: Num[NDArray, "17 1"]) -> Num[NDArray, "17 1"]:
    return -arr

sel = kps[0]
display(sel.shape)
display(sel)
sel = xz_ground_2_xy_ground(sel)
# reverse the upright axis
sel[:, 2] = -sel[:, 2]

scatters = [go.Scatter3d(x=[sel[j.index, 0]], y=[sel[j.index, 1]], z=[sel[j.index, 2]], mode='markers', marker=dict(size=CIRCLE_SIZE, color=j.color), name=j.name) for j in human_36_joints]
fig = go.Figure(data=scatters)

fig.show()

(17, 3)

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.1446696e-01, -6.1251954e-03,  1.9091105e-02],
       [-8.3640546e-02,  4.2381471e-01,  8.5035756e-02],
       [-4.8755914e-02,  8.2781178e-01,  2.5291616e-01],
       [ 1.1601043e-01,  2.0160456e-04,  3.1454870e-03],
       [ 1.0900587e-01,  4.2841870e-01,  5.6020476e-02],
       [ 8.8844068e-02,  8.1666046e-01,  2.5199789e-01],
       [-8.5484087e-03, -2.0692950e-01, -8.1414118e-02],
       [-3.0300356e-02, -4.4630206e-01, -1.8443397e-01],
       [-4.3630097e-02, -5.1236218e-01, -2.8008282e-01],
       [-4.3892395e-02, -6.0151565e-01, -2.5731069e-01],
       [ 1.3603930e-01, -4.2591548e-01, -1.5658578e-01],
       [ 2.0560208e-01, -1.9120884e-01, -1.4736680e-02],
       [ 1.8421590e-01,  5.2810434e-02, -8.4852569e-02],
       [-1.8512690e-01, -4.1894948e-01, -1.3574049e-01],
       [-2.2158432e-01, -1.7878255e-01,  2.4499021e-02],
       [-2.0875648e-01,  5.0539434e-02, -5.5248547e-02]], dtype=float32)